<a href="https://colab.research.google.com/github/antonpetkoff/identifier-suggestion/blob/master/notebooks/00-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import utilities
import os
import shutil
from subprocess import check_output

In [2]:
!nvidia-smi

Mon Jun 29 11:25:00 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
%env WORKSPACE_DIR=/content/gdrive/My Drive/src

# TODO: how can one read an environment variable?!?
%cd '/content/gdrive/My Drive/src'

env: WORKSPACE_DIR=/content/gdrive/My Drive/src
/content/gdrive/My Drive/src


In [5]:
timestamp = check_output(['date', '-u', "+%Y-%m-%dT%H-%M-%S"]).decode('utf-8').strip()

os.environ['PROJECT_DIR'] = os.path.join(
    os.environ['WORKSPACE_DIR'],
    f'identifier-suggestion-{timestamp}',
)

In [6]:
!git clone https://github.com/antonpetkoff/identifier-suggestion.git --depth 1 "${PROJECT_DIR}"

Cloning into '/content/gdrive/My Drive/src/identifier-suggestion-2020-06-29T11-25-25'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 112 (delta 1), reused 64 (delta 1), pack-reused 0
Receiving objects: 100% (112/112), 961.76 KiB | 12.49 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [7]:
os.chdir(os.environ['PROJECT_DIR'])

In [8]:
!pwd
!ls -l

/content/gdrive/My Drive/src/identifier-suggestion-2020-06-29T11-25-25
total 69
drwx------  4 root root  4096 Jun 29 11:25 data
-rw-------  1 root root 35149 Jun 29 11:25 LICENSE
drwx------  2 root root  4096 Jun 29 11:25 notebooks
-rw-------  1 root root  9807 Jun 29 11:25 README.md
drwx------  3 root root  4096 Jun 29 11:25 reports
drwx------  2 root root  4096 Jun 29 11:25 requirements
drwx------ 13 root root  4096 Jun 29 11:25 src
drwx------  3 root root  4096 Jun 29 11:25 vscode-extension


In [9]:
%tensorflow_version 2.x

import tensorflow
print(tensorflow.__version__)

2.2.0


In [10]:
# Google Colab has standard libraries like numpy, pandas, matplotlib and TF (of course) pre-installed
!pip install -r requirements/colab.txt

     |████████████████████████████████| 92kB 4.0MB/s 
     |████████████████████████████████| 1.4MB 8.2MB/s 
     |████████████████████████████████| 4.3MB 22.1MB/s 
     |████████████████████████████████| 225kB 44.2MB/s 
     |████████████████████████████████| 102kB 11.5MB/s 
     |████████████████████████████████| 102kB 11.2MB/s 
     |████████████████████████████████| 112kB 62.8MB/s 
     |████████████████████████████████| 460kB 68.1MB/s 
     |████████████████████████████████| 112kB 58.2MB/s 
     |████████████████████████████████| 133kB 44.4MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
     |████████████████████████████████| 71kB 9.5MB/s 
  Created wheel for pathos: filename=pathos-0.2.6-cp36-none-any.whl size=77673 sha256=e6889cf7403de0583be032a2a39ff43f9adce0631fd549cbd9d48343d20b4b4a
  Stored in directory: /root/.cache/pip/wheels/3a/e8/c8/04cdd0c4bc6fbce35f642fc004244228916daae74bb0f482da
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size

In [11]:
# provide secrets to the project, e.g. access to wandb
shutil.copy(
    os.path.join(os.environ['WORKSPACE_DIR'], 'secrets/.env'),
    os.environ['PROJECT_DIR']
)

'/content/gdrive/My Drive/src/identifier-suggestion-2020-06-29T11-25-25/.env'

In [12]:
# monkey-patch (mock) os.symlink to be a noop, because wandb.save() uses it, but it is not supported by Google Colab Notebooks
os.symlink = lambda *x: print('Executing mocked noop symlink with arguments', x)

In [13]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
from argparse import Namespace
from src.pipelines.baseline import run

params = {
  'dir_data': '../data/processed/subtoken/',
  'file_checkpoint_dir': 'models/checkpoints/baseline/',
  'dir_preprocessed_data': '../data/processed/seq2seq/',
  'max_input_length': 128,
  'max_output_length': 8,
  'input_vocab_size': 5000,
  'input_embedding_dim': 50,
  'output_vocab_size': 5000,
  'output_embedding_dim': 50,
  'latent_dim': 320,
  'learning_rate': 0.0001,
  'epochs': 50,
  'batch_size': 1024,
  'random_seed': 1,
  'eval_averaging': 'macro'
}

run(Namespace(**params))

Initializing logger


wandb: Wandb version 0.9.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Experiment parameters:  Namespace(batch_size=1024, dir_data='../data/processed/subtoken/', dir_preprocessed_data='../data/processed/seq2seq/', epochs=50, eval_averaging='macro', file_checkpoint_dir='models/checkpoints/baseline/', input_embedding_dim=50, input_vocab_size=5000, latent_dim=320, learning_rate=0.0001, max_input_length=128, max_output_length=8, output_embedding_dim=50, output_vocab_size=5000, random_seed=1)
Loading preprocessed files...
Loaded input vocabulary.
Loaded output vocabulary.
Loaded preprocessed files.
Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
EncoderEmbedding (Embedding) multiple                  250000    
_________________________________________________________________
EncoderLSTM (LSTM)           multiple                  474880    
Total params: 724,880
Trainable params: 724,880
Non-trainable params: 0
___________________________________________________

wandb: Visualizing heatmap.


epoch 1 evaluation on test data time: 20.33506417274475 sec
epoch evaluation:  {'epoch': 1, 'epoch_test_loss': <tf.Tensor: shape=(), dtype=float32, numpy=2.7514312>, 'test_rouge_1_p': 0.20947096523268396, 'test_rouge_1_r': 0.09218176261982063, 'test_rouge_1_f1': 0.12280970461266753, 'test_rouge_2_p': 0.005089962121212122, 'test_rouge_2_r': 0.0019047196293290037, 'test_rouge_2_f1': 0.002560663233096267, 'test_rouge_3_p': 0.0, 'test_rouge_3_r': 0.0, 'test_rouge_3_f1': 0.0, 'test_rouge_L_p': 0.2094646239177489, 'test_rouge_L_r': 0.0921792260938466, 'test_rouge_L_f1': 0.12280608100413322}


...

epoch 50 - batch 10 - loss 0.8413720726966858
epoch 50 - batch 20 - loss 0.8412604928016663
epoch 50 - batch 30 - loss 0.8603997826576233
epoch 50 - batch 40 - loss 0.8726649880409241
epoch 50 - batch 50 - loss 0.8261845707893372
epoch 50 - batch 60 - loss 0.8132057189941406
epoch 50 - batch 70 - loss 0.9069117307662964
epoch 50 - batch 80 - loss 0.8558676838874817
epoch 50 - batch 90 - loss 0.826498806476593
epoch 50 - batch 100 - loss 0.8402538299560547
epoch 50 - batch 110 - loss 0.8495314717292786
epoch 50 - batch 120 - loss 0.8620043992996216
epoch 50 - batch 130 - loss 0.8186782002449036
epoch 50 - batch 140 - loss 0.7844666242599487
epoch 50 - batch 150 - loss 0.8352819085121155
epoch 50 - batch 160 - loss 0.8527398705482483
epoch 50 - batch 170 - loss 0.8278715014457703
epoch 50 - batch 180 - loss 0.8106671571731567
epoch 50 - batch 190 - loss 0.8620635271072388
epoch 50 - batch 200 - loss 0.8371495008468628
epoch 50 - batch 210 - loss 0.8470035791397095
epoch 50 - batch 220 - 

wandb: Visualizing heatmap.


epoch 50 evaluation on test data time: 22.596935033798218 sec
epoch evaluation:  {'epoch': 50, 'epoch_test_loss': <tf.Tensor: shape=(), dtype=float32, numpy=1.3755338>, 'test_rouge_1_p': 0.5468542548411408, 'test_rouge_1_r': 0.519023340870439, 'test_rouge_1_f1': 0.5242579163655157, 'test_rouge_2_p': 0.28306044541396114, 'test_rouge_2_r': 0.27439926609848486, 'test_rouge_2_f1': 0.2757308049769494, 'test_rouge_3_p': 0.12065683340097404, 'test_rouge_3_r': 0.11664616308171, 'test_rouge_3_f1': 0.11728427047902494, 'test_rouge_L_p': 0.5449458814065401, 'test_rouge_L_r': 0.5174068189065401, 'test_rouge_L_f1': 0.5225517559838023}


wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.


Saving checkpoint...
Saved checkpoint: models/checkpoints/baseline/ckpt-18
Done saving model
